# Build Media Tool Demo

This notebook demonstrates the `build_media` tool, which creates growth media compositions from ModelSEED compound IDs.

## Key Features

- Build minimal or complete media from compound IDs
- Set default uptake rates and custom bounds
- Automatically resolve compound names from database
- Store media as MSMedia objects (COBRApy-compatible)
- Session-based storage with auto-generated IDs

## Setup

First, let's import the necessary modules and load the ModelSEED database.

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent.parent / "src"))

from gem_flux_mcp.database import load_compounds_database, load_reactions_database
from gem_flux_mcp.database.index import DatabaseIndex
from gem_flux_mcp.tools.media_builder import build_media, BuildMediaRequest
from gem_flux_mcp.storage.media import retrieve_media

In [ ]:
# Load ModelSEED database
db_dir = Path.cwd().parent.parent / "data" / "database"
compounds_df = load_compounds_database(db_dir / "compounds.tsv")
reactions_df = load_reactions_database(db_dir / "reactions.tsv")
db_index = DatabaseIndex(compounds_df, reactions_df)

print(f"✅ Loaded {len(compounds_df)} compounds")
print(f"✅ Loaded {len(reactions_df)} reactions")

## Example 1: Glucose Minimal Media

Create a minimal media with glucose as the carbon source.

In [ ]:
# Build glucose minimal media
request = BuildMediaRequest(
    compounds=[
        "cpd00027",  # D-Glucose
        "cpd00007",  # O2
        "cpd00001",  # H2O
        "cpd00009",  # Phosphate
        "cpd00011",  # CO2
        "cpd00013",  # NH3
        "cpd00067",  # H+
    ],
    default_uptake=100.0,
    custom_bounds={
        "cpd00027": (-5.0, 100.0),   # Glucose limited to 5 mmol/gDW/hr
        "cpd00007": (-10.0, 100.0),  # Oxygen available at 10 mmol/gDW/hr
    }
)

response = build_media(request, db_index)

print(f"Media ID: {response['media_id']}")
print(f"Media type: {response['media_type']}")
print(f"Number of compounds: {response['num_compounds']}")
print(f"\nCompounds:")
for cpd in response['compounds']:
    lower, upper = cpd['bounds']
    print(f"  - {cpd['id']}: {cpd['name']} ({lower:.1f}, {upper:.1f})")

## Example 2: Retrieve Media Object

The media is stored as a ModelSEEDpy MSMedia object, which is compatible with COBRApy models.

In [ ]:
# Retrieve the MSMedia object
media_obj = retrieve_media(response['media_id'])

print(f"Type: {type(media_obj).__name__}")
print(f"Media ID: {media_obj.id}")
print(f"Has get_media_constraints: {hasattr(media_obj, 'get_media_constraints')}")

# Get media constraints (for applying to models)
constraints = media_obj.get_media_constraints()
print(f"\nMedia constraints (first 5):")
for cpd_id, bounds in list(constraints.items())[:5]:
    print(f"  {cpd_id}: {bounds}")

## Example 3: Complex Media

Create a more complex media with additional nutrients.

In [ ]:
# Build complex media with amino acids
request = BuildMediaRequest(
    compounds=[
        "cpd00027",  # D-Glucose
        "cpd00007",  # O2
        "cpd00001",  # H2O
        "cpd00009",  # Phosphate
        "cpd00011",  # CO2
        "cpd00013",  # NH3
        "cpd00067",  # H+
        "cpd00023",  # L-Glutamate
        "cpd00033",  # Glycine
        "cpd00035",  # L-Alanine
        "cpd00039",  # L-Lysine
        "cpd00041",  # L-Aspartate
        "cpd00051",  # L-Arginine
        "cpd00054",  # L-Serine
        "cpd00060",  # L-Methionine
        "cpd00066",  # L-Phenylalanine
        "cpd00069",  # L-Tyrosine
    ],
    default_uptake=100.0,
    custom_bounds={
        "cpd00027": (-10.0, 100.0),  # More glucose for complex media
        "cpd00007": (-20.0, 100.0),  # More oxygen
    }
)

response = build_media(request, db_index)

print(f"Media ID: {response['media_id']}")
print(f"Media type: {response['media_type']}")
print(f"Number of compounds: {response['num_compounds']}")
print(f"\nFirst 10 compounds:")
for cpd in response['compounds'][:10]:
    print(f"  - {cpd['id']}: {cpd['name']}")

## Example 4: Using Predefined Media

The server includes 4 predefined media compositions (see `data/media/` directory):
- `LB`: Luria-Bertani broth
- `M9`: M9 minimal media
- `minimal_glucose`: Minimal media with glucose
- `complete`: Rich media

You can also build these manually:

In [ ]:
# Build M9-like minimal media
request = BuildMediaRequest(
    compounds=[
        "cpd00027",  # D-Glucose (carbon)
        "cpd00007",  # O2
        "cpd00001",  # H2O
        "cpd00009",  # Phosphate
        "cpd00011",  # CO2
        "cpd00013",  # NH3 (nitrogen)
        "cpd00067",  # H+
        "cpd00099",  # Cl-
        "cpd00058",  # Cu2+
        "cpd00034",  # Zn2+
        "cpd00149",  # Co2+
        "cpd00030",  # Mn2+
        "cpd00048",  # Sulfate
        "cpd00063",  # Ca2+
        "cpd00205",  # K+
        "cpd00971",  # Na+
        "cpd00254",  # Mg2+
        "cpd10515", # Fe2+
        "cpd00244",  # Ni2+
        "cpd00263",  # Cations
    ],
    default_uptake=100.0,
    custom_bounds={
        "cpd00027": (-10.0, 100.0),  # Glucose
        "cpd00007": (-20.0, 100.0),  # O2
    }
)

response = build_media(request, db_index)

print(f"Media ID: {response['media_id']}")
print(f"Media type: {response['media_type']}")
print(f"Number of compounds: {response['num_compounds']}")

## Summary

The `build_media` tool:

✅ Creates MSMedia objects from compound IDs  
✅ Validates all compound IDs against database  
✅ Supports custom uptake bounds  
✅ Auto-generates media IDs with timestamps  
✅ Stores media objects in session  
✅ Compatible with COBRApy models  

**Next Steps**: Use the created media with `run_fba` or `gapfill_model` tools!